# Convolutional Neural Network

[Download_Dataset](https://drive.google.com/file/d/1SzuyFzVBMxEG3hb5zRbdSWxY0JNdJZPP/view?usp=drive_link)

### Importing the libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [18]:
import numpy as np
from keras.preprocessing import image

In [22]:
import pandas as pd

In [2]:
tf.__version__

'2.5.0'

[Keras API Doc](https://keras.io/api/)

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255, #스케일링
                                  shear_range = 0/2, # 기울도 보고
                                  zoom_range=0.2, # 확대도 해보고
                                   horizontal_flip =True #뒤집어도 보고
                                  )
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64), # 크기가 너무 커지면 분석이 느려져
                                                 batch_size = 32, #배치에 얼마나 많은 이미지를 배치할 지
                                                 class_mode = 'binary') #고양이와 개만 구분할 거니까

Found 8000 images belonging to 2 classes.


In [41]:
X, y = training_set.next()
print(X.shape)
print(y.shape)

(32, 64, 64, 3)
(32,)


### Preprocessing the Test set

In [ ]:
# 확인을 위해 test set도 처리해줘야지.
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

In [42]:
X, y = test_set.next()
print(X.shape)
print(y.shape)

(32, 64, 64, 3)
(32,)


## Part 2 - Building the CNN

### Initialising the CNN

In [6]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters =32, # 정해진 공식은 없어
                              kernel_size =3,
                              activation = 'relu',
                              input_shape =[64,64,3])) #pixel 64x64, color(RGB), 맨 처음 레이어에만 입력

### Step 2 - Pooling

In [9]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, #풀링사이즈 = 2x2
                                 strides = 2)) #2칸씩 이동하녀 풀링

### Adding a second convolutional layer

In [10]:
# 2층으로 쌓자
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [11]:
cnn.add(tf.keras.layers.Flatten()) # 그냥 연결해주면 돼

### Step 4 - Full Connection

[about Dense Layer](https://wikidocs.net/192928)

In [ ]:
# my code like ann
'''
cnn.add(tf.keras.layers.Dense(uints = 2,
                             activation = 'softmax' 
                             ))
                             '''

In [13]:
# 이거 최종 결정 레이어 아니야. 연결하는 레이어야
# 이미지를 ANN 이런식으로 나오는 과정이지
# 찾아보니 convolution 과 polling에선 각 데이터들이 연결되지 않아. 독립적으로 있는 거야 그래서 그걸 연결해주는 신경망이 필요한거야.
cnn.add(tf.keras.layers.Dense(units = 128, #왜? 유닛이 많으면 정확도가 높아져
                             activation = 'relu' 
                             ))

### Step 5 - Output Layer

In [14]:
cnn.add(tf.keras.layers.Dense(units = 1, #이진분류할 거니까
                             activation = 'sigmoid' 
                             ))

## Part 3 - Training the CNN

### Compiling the CNN

In [15]:
cnn.compile(optimizer='adam',
           loss='binary_crossentropy',
           metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [16]:
cnn.fit(x = training_set,
        validation_data= test_set,
       epochs = 25)

Epoch 1/25
250/250 [==============================] - 37s 144ms/step - loss: 0.6682 - accuracy: 0.5920 - val_loss: 0.6526 - val_accuracy: 0.6330
Epoch 2/25
250/250 [==============================] - 34s 137ms/step - loss: 0.6144 - accuracy: 0.6589 - val_loss: 0.6287 - val_accuracy: 0.6460
Epoch 3/25
250/250 [==============================] - 33s 133ms/step - loss: 0.5766 - accuracy: 0.6989 - val_loss: 0.6442 - val_accuracy: 0.6380
Epoch 4/25
250/250 [==============================] - 33s 133ms/step - loss: 0.5397 - accuracy: 0.7286 - val_loss: 0.5066 - val_accuracy: 0.7570
Epoch 5/25
250/250 [==============================] - 35s 140ms/step - loss: 0.5145 - accuracy: 0.7496 - val_loss: 0.5189 - val_accuracy: 0.7545
Epoch 6/25
250/250 [==============================] - 36s 142ms/step - loss: 0.4911 - accuracy: 0.7629 - val_loss: 0.5204 - val_accuracy: 0.7390
Epoch 7/25
250/250 [==============================] - 36s 142ms/step - loss: 0.4775 - accuracy: 0.7720 - val_loss: 0.4761 - val_ac

## Part 4 - Making a single prediction

In [30]:
test_image.shape

(1, 64, 64, 3)

In [37]:
test_image = image.load_img(path = 'dataset/single_prediction/cat_or_dog_2.jpg', #1 :개, 2: 고양이
                           target_size = (64,64)) # 기존 변형한 형태로 만들어줘야지

#근데 왜 스케일은 필요없을까?
# 추가자료 - 아래처럼 하면 이미지를 정규화 할 수 있어.
# result = cnn.predict(test_image /255.0)


test_image = image.img_to_array(test_image) # 검사할 때는 배열형태로 해야해

print(f'just change to array shape : {test_image.shape}')
# 이전에 배치에 넣어ㄴ놨기때문에 배치로 만들어놔야해.
test_image =  np.expand_dims(test_image,axis=0)
print(f'after expend, shape : {test_image.shape}')
      

# r결과
result = cnn.predict(test_image/255.0) # 이렇게 했을 때 문제는 if 문에서 완전한 1이 나오지 않을 수 있다는 거??
print(f'result : {result}')
#ㅇ 인코딩
training_set.class_indices
if result[0][0] > 0.5 :
    prediction = 'dog'
else :
    prediction = 'cat'
    
print(prediction)

just change to array shape : (64, 64, 3)
after expend, shape : (1, 64, 64, 3)
result : [[0.02214101]]
cat
